In [14]:
import pandas as pd

# Load the validation rules from the CSV file (this is your dataset above)
schema_df = pd.read_csv("../data/variation_USGovt.csv")

# Assuming you have the actual data in another CSV
data = pd.read_csv("../data/variation_USGovt.csv")  # Replace with your actual data file

# Function to check validation
def validate_row(row, rules):
    errors = []
    for _, rule_row in rules.iterrows():
        name = rule_row['name']
        rule = rule_row['rule']

        try:
            # Prepare the rule for evaluation by replacing field names with their actual values
            # Handle %vin% operator (used as a custom operator for `%in%`)
            if '%vin%' in rule:
                # Convert to Python's 'in' operator
                lhs, rhs = rule.split('%vin%')
                lhs = lhs.strip()
                rhs = rhs.strip()
                result = eval(f"{row[lhs]} in {eval(rhs)}")
            else:
                # Replace the variable name with row['field']
                local_dict = {col: row[col] for col in row.index}
                result = eval(rule, {}, local_dict)
                
            if not result:
                errors.append(f"{name} failed validation: {rule}")
        except Exception as e:
            errors.append(f"{name} rule error: {e}")
    return errors

# Apply validation row by row
validation_results = data.apply(lambda row: validate_row(row, schema_df), axis=1)

# Print or save the results
for idx, errors in enumerate(validation_results):
    if errors:
        print(f"Row {idx} Errors:")
        for error in errors:
            print(f" - {error}")


Row 0 Errors:
 - DOI rule error: invalid syntax (<string>, line 1)
 - unique rule error: name 'is_unique' is not defined
 - Sample_ID rule error: name 'Sample_ID' is not defined
 - Subsample_ID rule error: invalid syntax (<string>, line 1)
 - Location rule error: invalid syntax (<string>, line 1)
 - Approximate_Lattitude rule error: invalid syntax (<string>, line 1)
 - Approximate_Longitude rule error: invalid syntax (<string>, line 1)
 - Source rule error: 'Source'
 - Date rule error: invalid syntax (<string>, line 1)
 - Concentration rule error: invalid syntax (<string>, line 1)
 - Concentration_Units rule error: invalid syntax (<string>, line 1)
 - Color_Transparent rule error: too many values to unpack (expected 2)
 - Color_Blue rule error: too many values to unpack (expected 2)
 - Color_Red rule error: too many values to unpack (expected 2)
 - Color_Brown rule error: too many values to unpack (expected 2)
 - Color_Green rule error: too many values to unpack (expected 2)
 - Color_O